## Run experiments

In [1]:
import lancedb

/home/angeline/miniconda3/envs/mlops-training/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uri = "../../data/lancedb-data/audio-lancedb"
db = lancedb.connect(uri)


In [3]:
import pandas as pd

def test_method(test_fn):
    queries_tbl = db.open_table("audio_example_queries")
    total_rows = queries_tbl.count_rows()
    song_num_actual = []
    song_num_retrieved = []

    conditions = [
        "(offset == 0) and (pitch_shift == 0) and (time_stretch == 1.0)",
        "(offset != 0) and (pitch_shift == 0) and (time_stretch == 1.0)",
        "(offset == 0) and (pitch_shift != 0) and (time_stretch == 1.0)",
        "(offset == 0) and (pitch_shift == 0) and (time_stretch != 1.0)",
        "(offset == 0) and (pitch_shift != 0) and (time_stretch != 1.0)",
        "(offset != 0) and (pitch_shift != 0) and (time_stretch != 1.0)",
    ]

    for condition in conditions:
        print(f"Running test for condition: {condition}")
        filtered_tbl = queries_tbl.search().where(condition).select(["song_num", "vector"])

        for _, row in filtered_tbl.to_pandas().iterrows():
            song_num_actual.append(row["song_num"])
            retrieved_info_list = test_fn(row["vector"]).to_pandas()

            song_num_retrieved.append([retrieved_info["song_num"] 
                                       for _, retrieved_info in retrieved_info_list.iterrows()])
    return song_num_actual, song_num_retrieved


def calculate_mrr(actual_songs, retrieved_songs):
    """
    Calculate Mean Reciprocal Rank (MRR) for a list of song retrievals.

    Parameters:
    actual_songs (list of int): A list of the actual song numbers.
    retrieved_songs (list of list of int): A list of lists, where each inner list contains retrieved song numbers.

    Returns:
    float: The Mean Reciprocal Rank (MRR) score.
    """
    reciprocal_ranks = []

    for actual, retrieved in zip(actual_songs, retrieved_songs):
        try:
            # Find the rank (1-indexed) of the actual song in the retrieved list
            rank = retrieved.index(actual) + 1
            reciprocal_ranks.append(1 / rank)
        except ValueError:
            # If the actual song is not in the retrieved list, reciprocal rank is 0
            reciprocal_ranks.append(0.0)

    # Calculate the mean of the reciprocal ranks
    return sum(reciprocal_ranks) / len(reciprocal_ranks)

In [4]:
db_tbl = db.open_table("audio_dataset")

def default_search(query_vector):
    return db_tbl.search(query_vector).limit(3)

In [5]:
actual, retrieved = test_method(default_search)

Running test for condition: (offset == 0) and (pitch_shift == 0) and (time_stretch == 1.0)
Running test for condition: (offset != 0) and (pitch_shift == 0) and (time_stretch == 1.0)
Running test for condition: (offset == 0) and (pitch_shift != 0) and (time_stretch == 1.0)
Running test for condition: (offset == 0) and (pitch_shift == 0) and (time_stretch != 1.0)
Running test for condition: (offset == 0) and (pitch_shift != 0) and (time_stretch != 1.0)
Running test for condition: (offset != 0) and (pitch_shift != 0) and (time_stretch != 1.0)


In [6]:
calculate_mrr(actual, retrieved)

0.16666666666666666

## Track with wandb

In [7]:
import wandb

wandb.init(
    # set the wandb project where this run will be logged
    project="children-song-dataset-retrieval",

    # track hyperparameters and run metadata
    config={
    "embedding": "none",
    "retrieval": "l2",
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: angeliney (angeliney-georgian). Use `wandb login --relogin` to force relogin


In [8]:
wandb.log({"mrr": calculate_mrr(actual, retrieved)})
wandb.finish()

mrr,▁
mrr,0.16667


## Try with feature extraction

In [9]:
audio = db_tbl.search().limit(1).select(["song_num", "vector"]).to_pandas().iloc[0]["vector"]

In [12]:
import librosa
import numpy as np

def extract_features(audio, sr=44100, aggregate="summary_stat"):
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

    # Extract Chroma features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    
    # Extract Mel-scaled spectrogram features
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)

    if aggregate == "summary_stat":
        # Aggregate the MFCCs across time
        mfccs_mean = np.mean(mfccs, axis=1)
        mfccs_std = np.std(mfccs, axis=1)
        mfcc_embedding = np.concatenate([mfccs_mean, mfccs_std])

        chroma_mean = np.mean(chroma, axis=1)
        chroma_std = np.std(chroma, axis=1)
        chroma_embedding = np.concatenate([chroma_mean, chroma_std])

        mel_spectrogram_mean = np.mean(mel_spectrogram, axis=1)
        mel_spectrogram_std = np.std(mel_spectrogram, axis=1)
        mel_spectrogram_embedding = np.concatenate([mel_spectrogram_mean, mel_spectrogram_std])
    
    else:
        # Flatten the MFCCs into a 1D array
        mfcc_embedding = mfccs.flatten()
        chroma_embedding = chroma.flatten()
        mel_spectrogram_embedding = mel_spectrogram.flatten()
   
    return np.concatenate([mfcc_embedding, chroma_embedding, mel_spectrogram_embedding])

# Example usage
feat = extract_features(audio, aggregate="full")
print(feat.shape)

(131886,)


In [ ]:
## Todo re-embed the data with the new features